In [ ]:
import jupedsim as jps
from shapely import Polygon
import pathlib

In [ ]:
walls = Polygon([
    (0, 0), (0, 10), (10, 10), (10, 7.4), (10.2, 7.4), (10.2, 10), (14.8, 10), (14.8, 6.1), (15, 6.1), (15, 10.2), (-0.2, 10.2),(-0.2, -0.2), (15, -0.2), (15, 3.9), (14.8, 3.9), (14.8, 0),(10.2, 0), (10.2, 2.6), (10, 2.6), (10, 0)
])
area_total = Polygon([(-0.2,-0.2), (15,-0.2), (15, 10.2), (-0.2,10.2)])
walls_of_A = walls
walkable_area = area_total.difference(walls_of_A)
exit_area = Polygon ([ (14.8, 4),(15, 4),(15, 6),(14.8, 6) ])

In [ ]:
spawning_area = Polygon([(0.1, 0.1), (5.1, 0.1), (5.1, 9.9), (0.1, 9.9)])
num_agents = 85
pos_in_spawning_area = jps.distributions.distribute_by_number(
    polygon=spawning_area,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.2,
    seed=1,
)

In [ ]:
trajectory_file = "area_A.sqlite"

simulation = jps.Simulation(
    model = jps.CollisionFreeSpeedModel(),
    geometry = walkable_area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

#exit_coords = [list(polygon.exterior.coords) for polygon in exit_area]
#flat_exit_coords = [coord for sublist in exit_coords for coord in sublist]
exit_id = simulation.add_exit_stage(exit_area)

journey_id = simulation.add_journey(jps.JourneyDescription([exit_id]))

for pos in pos_in_spawning_area:
    agent_parameters = jps.CollisionFreeSpeedModelAgentParameters(
            journey_id=journey_id,stage_id = exit_id,position = pos,
        )
    simulation.add_agent(agent_parameters)

while simulation.agent_count() > 0 and simulation.iteration_count() < 5000: 
    simulation.iterate()

dt = 0.05
simulation_time = simulation.iteration_count() * dt
print(f"Simulation time for Case A is : {simulation_time} seconds.")

In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=10)